In [2]:
pdf_path = 'pdf/Midshore Verra Registry Communication Agreement - Blue Source LLC_signed.pdf'

In [3]:
import pdfplumber

def curves_to_edges(cs):
    """See https://github.com/jsvine/pdfplumber/issues/127"""
    edges = []
    for c in cs:
        edges += pdfplumber.utils.rect_to_edges(c)
    return edges

# Import the PDF.
pdf = pdfplumber.open(pdf_path)

# Load the first page.
for i in range(len(pdf.pages)):
    p = pdf.pages[i]
# Table settings.
    ts = {
        "vertical_strategy": "explicit",
        "horizontal_strategy": "explicit",
        "explicit_vertical_lines": curves_to_edges(p.curves + p.edges),
        "explicit_horizontal_lines": curves_to_edges(p.curves + p.edges),
        "intersection_y_tolerance": 10,
    }

    # Get the bounding boxes of the tables on the page.
    try:
        bboxes = [table.bbox for table in p.find_tables(table_settings=ts)]
    except:
        print(i)
        break
        
    def not_within_bboxes(obj):
        """Check if the object is in any of the table's bbox."""
        def obj_in_bbox(_bbox):
            """See https://github.com/jsvine/pdfplumber/blob/stable/pdfplumber/table.py#L404"""
            v_mid = (obj["top"] + obj["bottom"]) / 2
            h_mid = (obj["x0"] + obj["x1"]) / 2
            x0, top, x1, bottom = _bbox
            return (h_mid >= x0) and (h_mid < x1) and (v_mid >= top) and (v_mid < bottom)
        return not any(obj_in_bbox(__bbox) for __bbox in bboxes)

    print("Text outside the tables:")
    # print(p.filter(not_within_bboxes).extract_text())

Text outside the tables:
1
